In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import math

# Função para ler o arquivo e contar os elementos do grafo
def contar_metricas(caminho_arquivo):
    with open(caminho_arquivo, "r", encoding="utf-8") as arquivo:
        linhas = arquivo.readlines()

    vertices = set()
    arestas  = set()
    arcos    = set()
    vertices_requeridos = set()
    arestas_requeridas  = set()
    arcos_requeridos    = set()
    secao = None

    for linha in linhas:
        linha = linha.strip()
        if linha.startswith("ReN."):
            secao = "ReN";   continue
        elif linha.startswith("ReE."):
            secao = "ReE";   continue
        elif linha.startswith("EDGE"):
            secao = "EDGE";  continue
        elif linha.startswith("ReA."):
            secao = "ReA";   continue
        elif linha.startswith("ARC"):
            secao = "ARC";   continue

        if not linha or secao is None:
            continue

        partes = linha.split("\t")
        try:
            if secao == "ReN":
                no      = int(partes[0].replace("N",""))
                demanda = int(partes[1])
                scusto  = int(partes[2])
                vertices_requeridos.add((no, (demanda, scusto)))
                vertices.add(no)

            elif secao in ["ReE", "EDGE"]:
                u, v     = int(partes[1]), int(partes[2])
                t_custo  = int(partes[3])
                aresta   = (min(u,v), max(u,v))
                arestas.add((aresta, t_custo))
                vertices.update([u,v])
                if secao == "ReE":
                    demanda = int(partes[4])
                    scusto  = int(partes[5])
                    arestas_requeridas.add((aresta, (t_custo, demanda, scusto)))

            elif secao in ["ReA", "ARC"]:
                u, v     = int(partes[1]), int(partes[2])
                t_custo  = int(partes[3])
                arco     = (u,v)
                arcos.add((arco, t_custo))
                vertices.update([u,v])
                if secao == "ReA":
                    demanda = int(partes[4])
                    scusto  = int(partes[5])
                    arcos_requeridos.add((arco, (t_custo, demanda, scusto)))
        except:
            continue

    return vertices, arestas, arcos, vertices_requeridos, arestas_requeridas, arcos_requeridos

# Função para desenhar o grafo com custos visíveis
def plotar_grafo_com_custos(vertices, arestas, arcos):
    plt.figure(figsize=(8,6))

    # Ordena os vértices e os distribui ao redor de um círculo
    vs = sorted(vertices)
    N  = len(vs)
    angulos = np.linspace(0, 2*np.pi, N, endpoint=False)
    posicoes = {v:(np.cos(a), np.sin(a)) for v,a in zip(vs, angulos)}

    # Desenha os nós
    for v, (x, y) in posicoes.items():
        plt.scatter(x, y, s=300, color='lightblue', edgecolors='black', zorder=3)
        plt.text(x, y, str(v), ha='center', va='center', zorder=4)

    # Desenha arestas com custo
    for (u, v), t_custo in arestas:
        x1, y1 = posicoes[u]
        x2, y2 = posicoes[v]
        plt.plot([x1, x2], [y1, y2], 'k-', lw=1, zorder=2)
        mx, my = (x1 + x2)/2, (y1 + y2)/2
        plt.text(mx, my, str(t_custo), color='black', fontsize=9, zorder=5)

    # Desenha arcos (direcionados) com custo
    for (u, v), t_custo in arcos:
        x1, y1 = posicoes[u]
        x2, y2 = posicoes[v]
        dx, dy = x2 - x1, y2 - y1
        plt.arrow(x1, y1, dx*0.85, dy*0.85,
                  head_width=0.05, head_length=0.05,
                  fc='red', ec='red', lw=1, zorder=2,
                  length_includes_head=True)
        mx, my = (x1 + x2)/2, (y1 + y2)/2
        plt.text(mx, my, str(t_custo), color='red', fontsize=9, zorder=5)

    plt.axis('off')
    plt.title("Visualização do Grafo com custos")
    plt.show()

# Entrada do caminho do arquivo pelo usuário
caminho_arquivo = input("Digite o caminho do arquivo .dat (ex: mgval_0.50_8B.dat): ")
vertices, arestas, arcos, v_req, e_req, a_req = contar_metricas(caminho_arquivo)
print(f"Carregado: {len(vertices)} vértices, {len(arestas)} arestas, {len(arcos)} arcos")

# Chamada para desenhar o grafo
plotar_grafo_com_custos(vertices, arestas, arcos)
